In [216]:
# Importa las librerias a usar
import pandas as pd
import re
import numpy as np
import funcionesw as fw
import datetime

importar arvhivos a procesar

In [217]:
# ubicacion de los archivos
patch1 = '../input/Kilometros en Servicio MIO - 05-Mar-2024.xlsx'# Se importa el archivo xlxs
patch2 =  '../input/REPORTE FINALIZADO 05_MARZ_2024.xlsx'
patch3 = '../input/JERARQUIAS.xlsx'

# Se importa el archivo xlxs
kmsmio = pd.read_excel(patch1, sheet_name='Programacion')
repofi = pd.read_excel(patch2, sheet_name='Programacion')
bitacora = pd.read_excel(patch2, sheet_name='Bitacora')
jerarquias = pd.read_excel(patch3, sheet_name='Hoja1')

# Se eliminan las filas y las columnas vacias
kmsmio = kmsmio.dropna(axis=0, how='all').dropna(axis=1, how='all')
repofi = repofi.dropna(axis=0, how='all').dropna(axis=1, how='all')
bitacora = bitacora.dropna(axis=0, how='all').dropna(axis=1, how='all')

# Se crea el df de adicionales
adi = kmsmio[kmsmio['Kilometros Programados Plan'].isna()]

Porcesar K2

In [218]:
# Filtra el DF por Kms en Servicio
kmsmio = kmsmio[kmsmio['Tipo Kilómetros']== 'Kms en Servicio']
kmsmio = kmsmio[kmsmio['Kilometros Programados Plan'].notna()]

# Se aplica la funcion extraer_hora_regex para estandarizar la hora
kmsmio['Desde'] = kmsmio['Desde'].apply(fw.extraer_hora_regex)
kmsmio['hasta'] = kmsmio['hasta'].apply(fw.extraer_hora_regex)

# Se crea una nueva columna 'id'
kmsmio['id'] = kmsmio.apply(lambda row: '-'.join((str(row['Fecha']), str(row['Línea']), str(row['Número de tarea vehículo']), 
                                                   str(row['Desde']), str(row['hasta']))), axis=1)

# Se verifica que la columna id no tenga id repetidos
kmsmio['id'].value_counts().sort_values(ascending=False)


id
2024-03-05 00:00:00-A14A-1-05:00:00-05:12:10        1
2024-03-05 00:00:00-T47B-2-19:21:48-20:01:10        1
2024-03-05 00:00:00-A14A-1-05:26:00-05:38:10        1
2024-03-05 00:00:00-A14A-1-05:38:55-05:51:15        1
2024-03-05 00:00:00-A14A-1-05:52:00-06:04:43        1
                                                   ..
2024-03-05 00:00:00-A35A-99753-24:03:04-24:14:39    1
2024-03-05 00:00:00-A35A-99753-23:03:04-23:14:39    1
2024-03-05 00:00:00-A35A-99753-21:33:04-21:44:07    1
2024-03-05 00:00:00-A35A-99753-21:30:02-21:30:17    1
2024-03-05 00:00:00-A35A-99753-24:16:10-24:29:23    1
Name: count, Length: 18645, dtype: int64

Procesar reporte finalizado

In [219]:
# Se aplica la funcion extraer_hora_regex para estandarizar la hora
repofi['desde'] = repofi['desde'].apply(fw.extraer_hora_regex)
repofi['hasta'] = repofi['hasta'].apply(fw.extraer_hora_regex)

# Filtra el DF por Kms en Servicio
repofi = repofi[repofi['Tipo de kilometros']== 'Kms en Servicio']

# Se crea una nueva columna 'id'
repofi['id'] = repofi.apply(lambda row: '-'.join((str(row['Fecha']), str(row['Línea']), str(row['Número de tarea vehículo']), 
                                                   str(row['desde']), str(row['hasta']))), axis=1)

# Se seleccionan las columnas del reporte finalizado
repofi = repofi[['id', 'Largo', 'Número de secuencia de carro', 'Número de tarea vehículo', 'Secuencia de arcos', 
                 'Número de viaje', 'Incumplidos','Descripción Novedad','Concesionario de Transporte', 'desde', 'hasta']]

# Se verifica que el id no tenga datos repetidos
repofi['id'].value_counts().sort_values(ascending=False)

id
2024-03-05 00:00:00-A14A-1-05:00:00-05:12:10        1
2024-03-05 00:00:00-T47B-2-19:21:48-20:01:10        1
2024-03-05 00:00:00-A14A-1-05:26:00-05:38:10        1
2024-03-05 00:00:00-A14A-1-05:38:55-05:51:15        1
2024-03-05 00:00:00-A14A-1-05:52:00-06:04:43        1
                                                   ..
2024-03-05 00:00:00-A35A-99753-24:03:04-24:14:39    1
2024-03-05 00:00:00-A35A-99753-23:03:04-23:14:39    1
2024-03-05 00:00:00-A35A-99753-21:33:04-21:44:07    1
2024-03-05 00:00:00-A35A-99753-21:30:02-21:30:17    1
2024-03-05 00:00:00-A35A-99753-24:16:10-24:29:23    1
Name: count, Length: 18645, dtype: int64

Se realiza el respectivo Merge con los dos df

In [220]:
# Se realiza el merge con outer en la columna id
merge = pd.merge(kmsmio, repofi, on='id', how='outer')

# Se verifica si hay duplicados o no coincidencias 
print(merge[merge.duplicated(subset='id')])

# Se verifica que el numero de filas en los df sea igual (True), para poder concluir que el merge fue exitoso
print(f'Len == en los 3 df: ', merge.shape[0] == kmsmio.shape[0] == repofi.shape[0])

Empty DataFrame
Columns: [Fecha, Día tipo, Designación de tarea vehículo, Línea, Tipo de viaje corto, Sentido, Tipo Kilómetros, Número de Vehículo, Concesionario de Transporte_x, Número de tarea vehículo_x, Desde, hasta_x, Duración, Punto de inicio, Punto de término, Largo_x, Número de viaje_x, Tipo de vehículo del viaje, Secuencia de arcos_x, Descripción Novedad_x, Concesionario Programado, Tipo de vehículo Programado, Incumplidos_x, Kilometros Programados-Desvios, Kilometros Programados Plan, Clase de viaje, a evaluar, TAREA, LINEA, VIAJE, VEHICULO SGCO, Kms en Servicio (MTS PLANEADOS), PARADAS PLANEADAS, PARADAS REALIZADAS SGCO, % PARADAS REALIZADAS SGCO, Kms en Servicio (MTS SGCO), FUENTE SGCO, HORA REAL SALIDA, HORA REAL LLEGADA, id, Largo_y, Número de secuencia de carro, Número de tarea vehículo_y, Secuencia de arcos_y, Número de viaje_y, Incumplidos_y, Descripción Novedad_y, Concesionario de Transporte_y, desde, hasta_y]
Index: []

[0 rows x 50 columns]
Len == en los 3 df:  True

Calificaciones de jerarquias en Bitacora

In [221]:
# Selecciona las columnas a usar
jerarquias = jerarquias[['JERARQUIA HGI RECO2',	'CALIFICACION']]

# Esto elimina los espacios en blanco adelante y atras
jerarquias['JERARQUIA HGI RECO2'] = jerarquias['JERARQUIA HGI RECO2'].str.strip()

#combina los el df de bitacora con el de jerarquias
bitacora = pd.merge(bitacora, jerarquias, left_on='Jerarquía', right_on='JERARQUIA HGI RECO2', how='left' )

# Esta linea es para verificar las filas que no se cruzaron por jerarquia y agregarlas a la matriz fe jererquia en input
x = bitacora[(bitacora['Jerarquía'].str.startswith('Salida'))& bitacora['CALIFICACION'].isnull()]
x['Jerarquía'].value_counts()

Series([], Name: count, dtype: int64)

In [222]:
# Este codigo elimina los duplicado de la columna calificaciones eliminando el que no afecta a kilometros

# Convertir la columna 'CALIFICACION' a numérica para poder comparar y encontrar el máximo
bitacora['CALIFICACION'] = pd.to_numeric(bitacora['CALIFICACION'], errors='coerce')

# Agrupar por las columnas 'Viaje' y 'Tarea', y mantener el valor máximo de 'CALIFICACION'
bitacora_grouped = bitacora.groupby(['Viaje', 'Tarea']).agg({'CALIFICACION': 'max'}).reset_index()

# Identificar las filas duplicadas en la tabla original
duplicados = bitacora[bitacora.duplicated(subset=['Viaje', 'Tarea'], keep=False)]

# Mantener solo una fila para cada grupo de duplicados en 'Viaje' y 'Tarea'
bitacora_sin_duplicados = bitacora.drop_duplicates(subset=['Viaje', 'Tarea'])

# Fusionar los datos filtrados con los valores máximos de 'CALIFICACION'
bitacora_final = pd.merge(bitacora_sin_duplicados, bitacora_grouped, on=['Viaje', 'Tarea'])

# Eliminar cualquiera de las filas duplicadas si ambas tienen valores nulos en 'CALIFICACION'
bitacora = bitacora_final.drop_duplicates(subset=['Viaje', 'Tarea'], keep='last', ignore_index=True)

# Se elimina la columna CALIFICACION_X y se renombra la columna CALIFICACION_Y
bitacora.drop(columns='CALIFICACION_x', axis=1, inplace=True)
bitacora.rename(columns={'CALIFICACION_y': 'CALIFICACION'}, inplace=True)

# Se verifica que no hay duplicados
bitacora[bitacora.duplicated(subset=['Viaje', 'Tarea'], keep=False)]

,# Caso MC,Tipo novedad,Jerarquía,Tarea,Viaje,Parada,Operador,Línea,Vehículo,Hora,...,Descripción,Jerarquia,Tipo de Novedad,Novedad,Retoma/Salida,Unnamed: 16,Unnamed: 17,Unnamed: 18,JERARQUIA HGI RECO2,CALIFICACION


Merge con bitacora


In [223]:
# Se reemplan las filas nulas por ceros de las dos columnas para posteriormente carbiar el tipo de dato
bitacora[['Tarea', 'Viaje']] = bitacora[['Tarea', 'Viaje']].fillna(0).astype(int)

# Se selecionan las columnas de Bitacora que se van a usar
bitacora = bitacora[['Jerarquía',	'Tarea',	'Viaje', 'Parada', 'Vehículo', 'Novedad', 'CALIFICACION']]
# Se filtran los casos que no tiene numero de viaje ya que no tiene retoma
bitacora = bitacora[bitacora['Viaje'] != 0]

# Se agrega al merge anterior la informacion de bitacora
# Importante tener en cuenta los vehiculos que tengan mas de un caso en la bitacora, esto genera id duplicados.
mergebitacora = pd.merge(merge, bitacora, left_on=['Número de tarea vehículo_x', 'Número de viaje_x'], right_on=['Tarea', 'Viaje'], how='outer' )

# Revisar este resultado, ya que los valores que no coinceden deben ser iguales en el filtro
print(f'len == en los 2 df ',mergebitacora.shape[0] ==  merge.shape[0])

len == en los 2 df  True


In [224]:
# Elimina las filas que 'Número de Vehículo' y 'Número de secuencia de carro' esten vacios
def eliminar_no_realizados(row):
    if pd.isna(row['Número de Vehículo']) and pd.isna(row['Número de secuencia de carro']):
        index = row.name
        mergebitacora.drop(index, axis=0, inplace=True) 
# Aplicar la función a cada fila del DataFrame mergebitacora
mergebitacora.apply(eliminar_no_realizados, axis=1)


# Cambia Blanco y Negro Masivo por BYN en la columna Concesionario Programado
mergebitacora['Concesionario Programado'] = mergebitacora['Concesionario Programado'].replace('Blanco y Negro Masivo','BYN')

In [225]:
# Condiciones para la columna condicional
condicion1 = mergebitacora['Largo_x'] != mergebitacora['Largo_y']
condicion2 = mergebitacora['Número de Vehículo'] != mergebitacora['Número de secuencia de carro']
condicion3 = mergebitacora['Secuencia de arcos_x'] != mergebitacora['Secuencia de arcos_y']
condicion4 = mergebitacora['Concesionario de Transporte_x'] != mergebitacora['Concesionario Programado']

# Aplicar condiciones y asignar valores a la nueva columna
mergebitacora['Condicional_ini'] = np.where(condicion1 & condicion2 & condicion3 & condicion4, 'Revisar largo, parada, vehículo y consecionario',
                                    np.where(condicion1 & condicion2 & condicion3, 'Revisar largo, parada y vehículo',
                                        np.where(condicion1 & condicion2 & condicion4, 'Revisar largo, vehiculo y consecionario',         
                                            np.where(condicion1 & condicion3 & condicion4, 'Revisar largo, parada y consecionario',
                                                np.where(condicion2 & condicion3 & condicion4, 'Revisar vehiculo, parada y consecionarios',        
                                                    np.where(condicion1 & condicion2, 'Revisar largo y vehiculo',
                                                        np.where(condicion1 & condicion3, 'Revisar largo y parada',
                                                            np.where(condicion1 & condicion4, 'Revisar largo y consecionario', 
                                                                np.where(condicion2 & condicion3, 'Revisar vehiculo y parada',
                                                                    np.where(condicion2 & condicion4, 'Revisar vehiculo y consecionario',        
                                                                        np.where(condicion3 & condicion4, 'Revisar parada y consecionario',                                            
                                                                            np.where(condicion1, 'Revisar largo', 
                                                                                np.where(condicion2, 'Revisar numero de vehiculo',
                                                                                    np.where(condicion3, 'Revisar paradas',
                                                                                        np.where(condicion4, 'Revisar consesionario', 'OK')))))))))))))))

In [226]:
# mergebitacora.to_excel('../output/prueba_4_columns.xlsx', sheet_name='Solicitud_M')

Revisar de numero de vehiculos para posteriormente verificar el incumplimiento

In [227]:
# Se crea una nueva columna donde se pondra los cambios que realizara la funcion
mergebitacora['cambios_realizados'] = np.nan

# Se aplica la funcion al df
mergebitacora['cambios_realizados'] = mergebitacora.apply(fw.vehiculos_xy_texto, axis=1)

# Se aplica la funcion vehiculo_xy
mergebitacora['Número_de_secuencia_de_carro_p'] = mergebitacora.apply(fw.vehiculo_xy, axis=1)

# Se verifica 
mergebitacora['cambios_realizados'].value_counts()

cambios_realizados
vehiculo_y Nan repl k2      9496
Numero ok                   6445
vehiculo_x != vehiculo_y     198
x=nan vihiculo_y no Nan      156
Name: count, dtype: int64

TESTEADO

In [228]:
# Se seleccionan las columnas de adicional para el merge
adi = adi[['Número de Vehículo','Desde','hasta']]

# Convertir la columna 'Desde' a tipo datetime
adi['Desde'] = pd.to_datetime(adi['Desde'], format='%H:%M:%S')
adi['hasta'] = pd.to_datetime(adi['hasta'], format='%H:%M:%S')

# Restar un minuto a cada valor en la columna 'Desde'
adi['Desde'] = adi['Desde'] - pd.Timedelta(minutes=1)
adi['hasta'] = adi['hasta'] - pd.Timedelta(minutes=1)

# Se aplica nuevamente la estandarizacion en la hora
adi['Desde'] = adi['Desde'].apply(fw.extraer_hora_regex)
adi['hasta'] = adi['hasta'].apply(fw.extraer_hora_regex)

# Crea una nueva columna llamada 'ID' que combine las columnas 'Número de secuencia de carro', 'Desde' y 'hasta_x'
mergebitacora['idadi'] = mergebitacora.apply(lambda row: f"{row['Número_de_secuencia_de_carro_p']}_{row['desde']}_{row['hasta_y']}", axis=1)
adi['idadi'] = adi.apply(lambda row: f"{row['Número de Vehículo']}_{row['Desde']}_{row['hasta']}", axis=1)

# Se verifican si hay duplicados por id en el df adicionales
print(adi[adi.duplicated(subset='idadi', keep=False)])# Preguntar a cristan

      Número de Vehículo     Desde     hasta                    idadi
20274              22117  16:10:09  16:39:28  22117_16:10:09_16:39:28
20275              22117  16:10:09  16:39:28  22117_16:10:09_16:39:28


In [229]:
# Merde de merbitacora y df adicional
merge_bi_adic = pd.merge(mergebitacora, adi, on='idadi', how='left')

# Revisar este resultado, ya que los valores que no coinceden deben ser iguales en el filtro # Cual vehiculo dejar o eliminar, revisar la descripcion
print(f'len == en los 2 df ',mergebitacora.shape[0] ==  merge_bi_adic.shape[0])


len == en los 2 df  True


ok - revisar adicioneles 

Revision de casos amarillos y rosa

In [230]:
# Seleccion de casos Amarillos
amarillos = ['INGRESO::RETOMA','SALIDAS::POR FLOTA','SALIDAS::POR ACCIDENTE','SALIDAS::POR ABANDONO DE TAREA','INGRESO::ADICIONAL',
            'SALIDAS::POR VANDALISMO','SALIDAS::POR ALARMA','CANCELACION::DE ADICIONAL','SALIDAS::POR OPERADOR'
            ]



In [231]:
# Esta funcion marcara en una columna los cambios que se hagan a la columna Largo_y
def rev_amarillos_rosa_text(row):
    """
    Esta función se aplica a cada fila del DataFrame para evaluar si se cumplen ciertas condiciones
    basadas en las columnas 'Número de Vehículo', 'Descripción Novedad_x' y 'PARADAS PLANEADAS'.
    Según las condiciones cumplidas, devuelve un texto descriptivo correspondiente.

    Parámetros:
    row (pandas.Series): Una fila del DataFrame que contiene los datos de la bitácora.

    Retorna:
    str: Un texto descriptivo basado en las condiciones especificadas.

    Detalles de la lógica:
    - Si 'Número de Vehículo' no es NaN y 'Descripción Novedad_x' no es NaN, se evalúan las condiciones.
    - Si 'Descripción Novedad_x' contiene al menos uno de los elementos de la lista 'amarillos',
      se procede a realizar búsquedas específicas en 'Descripción Novedad_x'.
    - Se utilizan expresiones regulares para buscar patrones relacionados con 'INGRESO', 'SALIDAS' y 'CANCELACION'
      seguidos del número de parada en 'Descripción Novedad_x'.
    - Dependiendo de los resultados de las búsquedas y de los valores en 'PARADAS PLANEADAS', se devuelve un texto
      descriptivo que indica cómo se debe manejar el valor de 'Largo' en esa fila.

    """
    if pd.notna(row['Número de Vehículo_x']):
        if pd.notna(row['Descripción Novedad_x']): 
            # Casos rosa
            if isinstance(row['Descripción Novedad_x'], str) and 'Nro. Caso Desvio:' in row['Descripción Novedad_x']:
                    if row['Largo_y'] == row['Kilometros Programados Plan']:
                        return 'Caso rosa1'
                    elif row['Largo_y'] != row['Kilometros Programados Plan']:
                        if row['Largo_y'] != row['Kilometros Programados-Desvios']:
                            if row['Largo_y'] < row['Largo_x']:
                                return 'Caso rosa2'
                            else:
                                return 'Caso rosa2 else'
                        else:
                            'Caso rosa else 2'
                    else:
                        return 'Caso rosa else'
            # Casos Amarillos                
            elif isinstance(amarillos, list) and any(isinstance(amarillo, str) and amarillo in row['Descripción Novedad_x'] for amarillo in amarillos):
                ingreso_match = re.search(r'INGRESO.*?Nro\. Parada:\s*(\d{1,3})', row['Descripción Novedad_x'])
                salida_match = re.search(r'SALIDAS.*?Nro\. Parada:\s*(\d{1,3})', row['Descripción Novedad_x'])
                cancelacion_match = re.search(r'CANCELACION.*?Nro\. Parada:\s*(\d{1,3})', row['Descripción Novedad_x'])       
                if ingreso_match and int(ingreso_match.group(1)) == 1 and int(row['PARADAS PLANEADAS']) != 1:
                    return 'Caso amarillo1'
                elif ingreso_match and int(ingreso_match.group(1)) != 1:
                    return 'Caso amarillo2'
                elif salida_match and int(salida_match.group(1)) != int(row['PARADAS PLANEADAS']):
                    return 'Caso amarillo3'
                elif salida_match and int(salida_match.group(1)) == int(row['PARADAS PLANEADAS']):
                    return 'Caso amarillo4'
                elif cancelacion_match and int(cancelacion_match.group(1)) != int(row['PARADAS PLANEADAS']):
                    return 'Caso amarillo5'
                elif cancelacion_match and int(cancelacion_match.group(1)) == int(row['PARADAS PLANEADAS']):
                    return 'Caso amarillo6'
                else:
                    return 'Caso amarillo else'                       
            else:
                return 'Sin color'
        else:
            return 'Sin descripcion' 
    else:
        if pd.notna(row['Desde_y']):
            return  'casos verde' # No hay Número de Vehículo_x pero si info en Desde_y
        else:
            return 'No clasificado' # No hay Número de Vehículo_x


In [232]:
merge_bi_adic['texto_colores'] = np.nan

In [233]:
merge_bi_adic['texto_colores'] = merge_bi_adic.apply(rev_amarillos_rosa_text, axis=1)

In [234]:
merge_bi_adic['texto_colores'].value_counts()

texto_colores
Sin descripcion    14954
Caso rosa1           418
Caso amarillo4       311
Caso amarillo1       189
Caso amarillo2       166
No clasificado       148
Caso amarillo3        63
Sin color             30
casos verde            8
Caso amarillo6         6
Caso rosa2             1
Caso amarillo5         1
Name: count, dtype: int64

In [235]:
# Esta funcion marcara en una columna los cambios que se hagan a la columna Largo_y
def rev_amarillos_rosa(row):
    """
    Aplica lógica condicional para marcar los cambios realizados en la columna 'Largo_y' en función de las características de la fila.

    Parámetros:
    row (pandas.Series): Una fila del DataFrame que contiene los datos de la bitácora.

    Retorna:
    int or float: El valor de 'Largo_y' después de aplicar las modificaciones, o el mismo valor si no se realizan cambios.

    Detalles de la lógica:
    - Verifica si el 'Número de Vehículo' no es NaN.
    - Verifica si la 'Descripción Novedad_x' no es NaN.
    - Si la 'Descripción Novedad_x' contiene 'Nro. Caso Desvio:', aplica la lógica específica para estos casos:
      - Si 'Largo_y' es igual a 'Kilometros Programados Plan', devuelve 'Kilometros Programados-Desvios'.
      - Si 'Largo_y' es diferente a 'Kilometros Programados Plan', pero igual a 'Kilometros Programados-Desvios', devuelve 'Largo_y'.
      - Si 'Largo_y' es menor que 'Largo_x', devuelve 'Largo_x', de lo contrario, devuelve 'Largo_y'.
    - Si la 'Descripción Novedad_x' contiene alguna de las cadenas definidas en la lista 'amarillos', aplica la lógica específica para estos casos:
      - Basado en patrones de expresiones regulares, determina si se deben realizar cambios en 'Largo_y'.
      - Si se cumplen las condiciones anteriores, devuelve 'Largo_x' o 'Largo_y' según corresponda.
    - Si ninguna de las condiciones anteriores se cumple, devuelve el valor actual de 'Largo_y'.

    Casos Especiales:
    - Si el 'Número de Vehículo' es NaN, devuelve el valor actual de 'Largo_y'.
    """
    # Lógica condicional para marcar los cambios en 'Largo_y' según las características de la fila
      
    if pd.notna(row['Número de Vehículo_x']):
        if pd.notna(row['Descripción Novedad_x']): 
            # Casos rosa
            if isinstance(row['Descripción Novedad_x'], str) and 'Nro. Caso Desvio:' in row['Descripción Novedad_x']:
                    if row['Largo_y'] == row['Kilometros Programados Plan']:# Verificar el valor de los desvios # todo desvio debe salir el largo real de la medicion de los desvis que debe hacer el analista
                        return row['Kilometros Programados-Desvios'] # 'Caso rosa1'
                    elif row['Largo_y'] != row['Kilometros Programados Plan']:
                        if row['Largo_y'] != row['Kilometros Programados-Desvios']:
                            if row['Largo_y'] < row['Largo_x']:
                                return row['Largo_x'] ## 'Caso rosa2'
                            else:
                                row['Largo_y']
                        else:
                            row['Largo_y']
                    else:
                        row['Largo_y']
            # Casos Amarillos   # El largo debe salir del numero de parada del archivo de arcos cortos comerciales             
            elif isinstance(amarillos, list) and any(isinstance(amarillo, str) and amarillo in row['Descripción Novedad_x'] for amarillo in amarillos):
                ingreso_match = re.search(r'INGRESO.*?Nro\. Parada:\s*(\d{1,3})', row['Descripción Novedad_x'])
                salida_match = re.search(r'SALIDAS.*?Nro\. Parada:\s*(\d{1,3})', row['Descripción Novedad_x'])
                cancelacion_match = re.search(r'CANCELACION.*?Nro\. Parada:\s*(\d{1,3})', row['Descripción Novedad_x'])       
                if ingreso_match and int(ingreso_match.group(1)) == 1 and int(row['PARADAS PLANEADAS']) != 1:
                    return row['Largo_x']
                elif ingreso_match and int(ingreso_match.group(1)) != 1:
                    return row['Largo_x']
                elif salida_match and int(salida_match.group(1)) != int(row['PARADAS PLANEADAS']):
                    return row['Largo_x']
                elif salida_match and int(salida_match.group(1)) == int(row['PARADAS PLANEADAS']):
                    return  row['Largo_y']
                elif cancelacion_match and int(cancelacion_match.group(1)) != int(row['PARADAS PLANEADAS']):
                    return row['Largo_x']
                elif cancelacion_match and int(cancelacion_match.group(1)) == int(row['PARADAS PLANEADAS']):
                    return  row['Largo_y']
                else:
                    return  row['Largo_y']                       
            else:
                row['Largo_y']
        else:
            return row['Largo_y']
    else: 
        if pd.notna(row['Desde_y']):
            return  row['Largo_x']
        else:
            row['Largo_y']
        

In [236]:
merge_bi_adic['Largo_p'] = merge_bi_adic.apply(rev_amarillos_rosa, axis=1)

In [237]:
# Elimina el numero de vehiculo en los casos verdes
def delete_Vahi_caso_verde(row):
    if pd.notna(row['Desde_y']):# Verificar la secuencia del carro # no se p
        return np.nan

merge_bi_adic['Número de secuencia de carro_p'] = merge_bi_adic.apply(delete_Vahi_caso_verde, axis=1)


In [238]:
# Actualiza la condicion a la nueva columna 
condicion1 = merge_bi_adic['Largo_x'] != merge_bi_adic['Largo_p']
condicion2 = merge_bi_adic['Número de Vehículo_x'] != merge_bi_adic['Número_de_secuencia_de_carro_p']
condicion3 = merge_bi_adic['Secuencia de arcos_x'] != merge_bi_adic['Secuencia de arcos_y']
condicion4 = merge_bi_adic['Concesionario de Transporte_x'] != merge_bi_adic['Concesionario Programado']

# Aplicar condiciones y asignar valores a la nueva columna
merge_bi_adic['Condicional_fin'] = np.where(condicion1 & condicion2 & condicion3 & condicion4, 'Revisar largo, parada, vehículo y consecionario',
                                    np.where(condicion1 & condicion2 & condicion3, 'Revisar largo, parada y vehículo',
                                        np.where(condicion1 & condicion2 & condicion4, 'Revisar largo, vehiculo y consecionario',         
                                            np.where(condicion1 & condicion3 & condicion4, 'Revisar largo, parada y consecionario',
                                                np.where(condicion2 & condicion3 & condicion4, 'Revisar vehiculo, parada y consecionarios',        
                                                    np.where(condicion1 & condicion2, 'Revisar largo y vehiculo',
                                                        np.where(condicion1 & condicion3, 'Revisar largo y parada',
                                                            np.where(condicion1 & condicion4, 'Revisar largo y consecionario', 
                                                                np.where(condicion2 & condicion3, 'Revisar vehiculo y parada',
                                                                    np.where(condicion2 & condicion4, 'Revisar vehiculo y consecionario',        
                                                                        np.where(condicion3 & condicion4, 'Revisar parada y consecionario',                                            
                                                                            np.where(condicion1, 'Revisar largo', 
                                                                                np.where(condicion2, 'Revisar numero de vehiculo',
                                                                                    np.where(condicion3, 'Revisar paradas',
                                                                                        np.where(condicion4, 'Revisar consesionario', 'OK')))))))))))))))

In [239]:
merge_bi_adic.to_excel('../output/despuesprueba2.xlsx')

In [54]:
# x = mergebitacora[mergebitacora['Desde_y'].notna()]
# x[['Número de secuencia de carro','Número de Vehículo_x','Largo_x', 'Largo_y']]

In [55]:
# # Esta funcion en caso de tener adicioneales realiza el siguiente proceso
# def verdes(row):
#     if pd.notna(row['Desde_y']):
#         row['Incumplidos_y'] = row['Incumplidos_x']



In [56]:
# # Elimina las columnas que no son necesarias
# mergebitacora.drop(columns=['Número de Vehículo_y', 'Desde_y', 'hasta', 'texto'], inplace= True)

In [57]:
mergebitacora['Incumplidos_x'].value_counts()

Incumplidos_x
2.0    331
0.0     68
1.0     62
3.0      4
Name: count, dtype: int64

ok

Calificaciones incumplimineto 

In [58]:
print(kmsmio['Incumplidos'].value_counts().sum())
print(repofi['Incumplidos'].value_counts().sum())
print(merge['Incumplidos_x'].value_counts().sum())
print(merge['Incumplidos_y'].value_counts().sum())
print(mergebitacora['Incumplidos_x'].value_counts().sum())
print(mergebitacora['Incumplidos_y'].value_counts().sum())

2815
1003
2815
1003
465
66


In [59]:
# Funcion replica de la funcion de Excel para el calculo de incumpliminetos, pero la siguiente funcion la anula, revisar si es necesaria
def col_calificaciones_funtion(row, df):
    """
    Calcula el valor del incumplimiento según las condiciones especificadas.

    Parámetros:
    row (pandas.Series): Una fila del DataFrame que contiene los datos de la bitácora.
    df (pandas.DataFrame): DataFrame que contiene los datos de la bitácora.

    Retorna:
    int or float or np.nan: El valor del incumplimiento calculado según las condiciones.
    """
    
    # Obtener el índice de la fila actual
    index = row.name
    
    # Calcular el índice de la fila siguiente
    next_index = index + 1
    next_index_next = index + 2
    
    # Obtener la fila siguiente si el índice es válido, de lo contrario, establecerla como None
    next_row = df.iloc[next_index] if next_index < len(df) else None
    next_row_next = df.iloc[next_index_next] if next_index_next < len(df) else None

    if pd.notnull(row['Incumplidos_y']):
        if row['Tipo Kilómetros'] == "Kms en Servicio" and pd.isna(row['Número de secuencia de carro']):
            if row['Tipo de viaje corto'] == "Viaje comercial":
                return 2
            elif row['Tipo de viaje corto'] in ["Viaje de punto operacional", "Viaje de posicionamiento"]:
                if next_row is not None and pd.isna(next_row['Número de secuencia de carro']):
                    if (next_row['Incumplidos_y'] == 2) and (row['Largo_y'] in [0, 1]):
                        return 1
                    elif (row['Largo_y'] not in [0, 1]) or (next_row['Largo_y']  in [0, 1]):
                        if pd.notna(next_row_next['Incumplidos_y']):
                            return next_row_next['Incumplidos_y'] 
                        else:
                            return 2
                    else:
                        return next_row['Incumplidos_y']
                else:
                    if row is not None and row['Largo_y'] in [0, 1]:
                        return 1
                    else:
                        return 2
            else:
                return np.nan
        elif (row['Número de secuencia de carro'] in ["CAMBIO", "CANCELADA"]) and row['Tipo Kilómetros'] == "Kms en Vacio":
            return 0
        else:
            return np.nan
    else:
        return row['Incumplidos_y']


In [60]:
# def col_calificaciones_function(row, df):
#     """
#     Calcula el valor del incumplimiento según las condiciones especificadas.

#     Parámetros:
#     row (pandas.Series): Una fila del DataFrame que contiene los datos de la bitácora.
#     df (pandas.DataFrame): DataFrame que contiene los datos de la bitácora.

#     Retorna:
#     int or float or np.nan: El valor del incumplimiento calculado según las condiciones.
#     """
    
#     # Obtener el índice de la fila actual
#     index = row.name
    
#     # Calcular el índice de la fila siguiente y la siguiente siguiente
#     next_index = index + 1
#     next_index_next = index + 2
    
#     # Obtener la fila siguiente si el índice es válido, de lo contrario, establecerla como None
#     next_row = df.iloc[next_index] if next_index < len(df) else None
#     next_row_next = df.iloc[next_index_next] if next_index_next < len(df) else None

#     # Verificar si hay valores nulos en 'Incumplidos_y'
#     if pd.notnull(row['Incumplidos_y']):
#         # Verificar condiciones específicas
#         if row['Tipo Kilómetros'] == "Kms en Servicio" and pd.isna(row['Número de secuencia de carro']):
#             if row['Tipo de viaje corto'] == "Viaje comercial":
#                 return 2
#             elif row['Tipo de viaje corto'] in ["Viaje de punto operacional", "Viaje de posicionamiento"]:
#                 if next_row is not None and pd.isna(next_row['Número de secuencia de carro']):
#                     if (next_row['Incumplidos_y'] == 2) and (row['Largo_y'] in [0, 1]):
#                         return 1
#                     elif (row['Largo_y'] not in [0, 1]) or (next_row['Largo_y']  in [0, 1]):
#                         if pd.notna(next_row_next['Incumplidos_y']):
#                             return next_row_next['Incumplidos_y'] 
#                         else:
#                             return 2
#                     else:
#                         return next_row['Incumplidos_y']
#                 else:
#                     if row is not None and row['Largo_y'] in [0, 1]:
#                         return 1
#                     else:
#                         return 2
#             else:
#                 return np.nan
#         elif (row['Número de secuencia de carro'] in ["CAMBIO", "CANCELADA"]) and row['Tipo Kilómetros'] == "Kms en Vacio":
#             return 0
#         else:
#             return np.nan
#     else:
#         return row['Incumplidos_y']


In [61]:
# #Aplicar la función a todo el DataFrame
# mergebitacora['Incumplido_y'] = mergebitacora.apply(col_calificaciones_funtion, axis=1, args=(mergebitacora,))

In [62]:
# print(kmsmio['Incumplidos'].value_counts().sum())
# print(repofi['Incumplidos'].value_counts().sum())
# print(merge['Incumplidos_x'].value_counts().sum())
# print(merge['Incumplidos_y'].value_counts().sum())
# print(mergebitacora['Incumplidos_x'].value_counts().sum())
# print(mergebitacora['Incumplidos_y'].value_counts().sum())

In [63]:
print(mergebitacora['Incumplidos_x'].value_counts().sum())
print(mergebitacora['Incumplidos_y'].value_counts().sum())

465
66


In [64]:
x = mergebitacora[(mergebitacora['Incumplidos_x']!=mergebitacora['Incumplidos_y']) & (pd.notna(mergebitacora['Incumplidos_x']))]

In [65]:
x[['Número de Vehículo_x', 'Incumplidos_x', 'Largo_x', 'Número de secuencia de carro', 'Incumplidos_y', 'Largo_y']]# los caso vacios son por los verdes

,Número de Vehículo_x,Incumplidos_x,Largo_x,Número de secuencia de carro,Incumplidos_y,Largo_y
176,22125,0.0,4965,None,NaN,4965.0
319,NaN,2.0,4704,None,NaN,NaN
321,NaN,2.0,4704,None,NaN,NaN
322,NaN,2.0,4704,None,NaN,NaN
323,NaN,2.0,4704,None,NaN,NaN
...,...,...,...,...,...,...
16091,NaN,2.0,3658,None,NaN,NaN
16092,NaN,2.0,4172,None,NaN,NaN
16093,54006,2.0,3412,None,NaN,3412.0
16097,NaN,2.0,384,None,NaN,NaN


In [66]:
mergebitacora['Incumplidos_y'].value_counts()

Incumplidos_y
0.0    38
2.0    28
Name: count, dtype: int64

In [67]:
def col_calificaciones(row, df):
    """
    Calcula el valor del incumplimiento según las condiciones especificadas.

    Parámetros:
    row (pandas.Series): Una fila del DataFrame que contiene los datos de la bitácora.
    df (pandas.DataFrame): DataFrame que contiene los datos de la bitácora.

    Retorna:
    int or float or np.nan: El valor del incumplimiento calculado según las condiciones.
    """
   
    if pd.isna(row['Incumplidos_x']) and pd.notna(row['Incumplidos_y']):
        return np.nan
    elif row['Incumplidos_x'] == 0:
        return 0
    elif row['Incumplidos_x'] in [2,3] and row['CALIFICACION'] == 1:
        return row['CALIFICACION']
    elif row['Incumplidos_x'] == 3:
        return row['Incumplidos_x']
    elif row['Incumplidos_x'] in [0,1,3] and row['Incumplidos_y']== 2:
        return row['Incumplidos_x'] 
    elif row['Incumplidos_x']== 2 and  row['Incumplidos_y']== 2:
        return row['Incumplidos_x'] 
    else:
        'verifica'


In [68]:
# Aplicar la función a todo el DataFrame
mergebitacora['Incumplidos_y'] = mergebitacora.apply(col_calificaciones, axis=1, args=(mergebitacora,))

In [69]:
mergebitacora['Incumplidos_y'].value_counts()

Incumplidos_y
0.0    68
2.0    11
1.0     7
3.0     2
Name: count, dtype: int64

In [70]:
mergebitacora.shape

(16295, 63)

OK pendiente revisar calificaciones

In [71]:
z = mergebitacora[(mergebitacora['Incumplidos_x'].isna())&(mergebitacora['Incumplidos_y'].notna())]

In [72]:
z[['VIAJE', 'TAREA', 'Incumplidos_x', 'Incumplidos_y']]

,VIAJE,TAREA,Incumplidos_x,Incumplidos_y


In [73]:
mergebitacora[['VIAJE', 'TAREA', 'Incumplidos_x', 'Incumplidos_y']]

,VIAJE,TAREA,Incumplidos_x,Incumplidos_y
0,1.0,1.0,NaN,NaN
1,2.0,1.0,NaN,NaN
2,9.0,1.0,NaN,NaN
3,10.0,1.0,NaN,NaN
4,17.0,1.0,NaN,NaN
...,...,...,...,...
16290,9955.0,99753.0,NaN,NaN
16291,9956.0,99753.0,NaN,NaN
16292,9957.0,99753.0,NaN,NaN
16293,9958.0,99753.0,NaN,NaN


verificacion

In [74]:
# ##
# ##
# ##no va
# ##
# ##
# # Definir una función para aplicar a cada grupo
# def eliminar_duplicados_bitacora(row):
#     # Verificar si hay al menos un valor en 'Incumplidos_x'
#     if row['Incumplidos_x'].isnull() and row['Incumplidos_x'].isnull():   
#         return row[row['Incumplidos_x'].notnull()]  # Conservar la fila con 'Incumplidos_x' no nulo
#     # Verificar si hay al menos un valor en 'Incumplidos_y'
#     elif row['Incumplidos_y'].notnull().any():
#         return row[row['Incumplidos_y'].notnull()]  # Conservar la fila con 'Incumplidos_y' no nulo
#     # Conservar la fila con el valor más bajo en 'CALIFICACION'
#     else:
#         return row.loc[row['CALIFICACION'].idxmin()]

In [75]:
# Muestra los id duplicados producto del merge con bitacora
mergebitacora[mergebitacora['id'].duplicated()]

,Fecha,Día tipo,Designación de tarea vehículo,Línea,Tipo de viaje corto,Sentido,Tipo Kilómetros,Número de Vehículo_x,Concesionario de Transporte_x,Número de tarea vehículo_x,...,Parada,Vehículo,Novedad,CALIFICACION,texto_cmb_vehi,idadi,Número de Vehículo_y,Desde_y,hasta,texto_colores


Revision de Adicionales

Columna condicional 

In [78]:
# Se cambian los vehiculos del cierre por los de k2##
mergebitacora['Número de secuencia de carro'] = [x if pd.notna(x) else y for x, y in zip(mergebitacora['Número de secuencia de carro'], mergebitacora['Número de Vehículo_x'])]
# Marca en la columnas Cambios los cambios que realiza el código
mergebitacora['condicion'] = [ np.nan if  pd.notna(x) else 'Se Cambia # V RF' for x, y in zip(mergebitacora['Número de secuencia de carro'], mergebitacora['Número de Vehículo_x'])]


In [79]:
# # Función para actualizar el texto de la columna condicion para Largo_y si Largo_x es mayor
def actualizar_largo_texto(row):
    if pd.isna(row['Largo_y']) and pd.isna(row['condicion']) :
        return 'Cambia largo por NAN'
    elif pd.isna(row['Largo_y']) and pd.notnull(row['condicion']) :
        return row['condicion'] + '/Cambia largo por NAN'
    elif row['Largo_x'] > row['Largo_y'] and pd.isna(row['condicion']):
        return 'Cambia x Mayor largo'
    elif row['Largo_x'] > row['Largo_y'] and pd.notnull(row['condicion']) :
        return row['condicion'] + '/Cambia x Mayor largo'
    else:
        if pd.notnull(row['condicion']):
            return row['condicion']
            

In [80]:
# # Función para actualizar los valores de Largo_y si Largo_x es mayor
# def actualizar_largo(row):
#     if pd.isna(row['Largo_y']):
#         return row['Largo_x']
#     elif row['Largo_x'] > row['Largo_y']:
#         return row['Largo_x']
#     else:
#         return row['Largo_y']

In [ ]:
# # Marcar las filas donde se realizan cambios en una nueva columna
# mergebitacora['condicion'] = mergebitacora.apply(actualizar_largo_texto, axis=1)

In [ ]:
# # Aplicar la función a la columna Largo_y
# mergebitacora['Largo_y'] = mergebitacora.apply(actualizar_largo, axis=1)

In [81]:
#Se verifican los cambios realizados
mergebitacora['condicion'].value_counts()

condicion
Se Cambia # V RF    156
Name: count, dtype: int64

In [82]:
'Número de secuencia de carro', 'Secuencia de arcos', 'Concesionario Programado'

('Número de secuencia de carro',
 'Secuencia de arcos',
 'Concesionario Programado')

In [76]:
# Aplicar condiciones y asignar valores a la nueva columna
mergebitacora['Condicional_fin'] = np.where(condicion1 & condicion2 & condicion3 & condicion4, 'Revisar largo, parada, vehículo y consecionario',
                                    np.where(condicion1 & condicion2 & condicion3, 'Revisar largo, parada y vehículo',
                                        np.where(condicion1 & condicion2 & condicion4, 'Revisar largo, vehiculo y consecionario',         
                                            np.where(condicion1 & condicion3 & condicion4, 'Revisar largo, parada y consecionario',
                                                np.where(condicion2 & condicion3 & condicion4, 'Revisar vehiculo, parada y consecionarios',        
                                                    np.where(condicion1 & condicion2, 'Revisar largo y vehiculo',
                                                        np.where(condicion1 & condicion3, 'Revisar largo y parada',
                                                            np.where(condicion1 & condicion4, 'Revisar largo y consecionario', 
                                                                np.where(condicion2 & condicion3, 'Revisar vehiculo y parada',
                                                                    np.where(condicion2 & condicion4, 'Revisar vehiculo y consecionario',        
                                                                        np.where(condicion3 & condicion4, 'Revisar parada y consecionario',                                            
                                                                            np.where(condicion1, 'Revisar largo', 
                                                                                np.where(condicion2, 'Revisar numero de vehiculo',
                                                                                    np.where(condicion3, 'Revisar paradas',
                                                                                        np.where(condicion4, 'Revisar consesionario', 'OK')))))))))))))))

In [85]:
mergebitacora['condicional2'].value_counts()

condicional2
Revisar largo                            9615
Revisar largo y consecionario            5410
OK                                        648
Revisar consesionario                     466
Revisar largo, parada y consecionario     156
Name: count, dtype: int64

In [86]:
# Exporta a un archivo excel el Df
# mergebitacora.to_excel('../output/prueba_mergebita.xlsx', index=False, sheet_name='k2_procesado')